# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
vacationDF = pd.read_csv("weatherdata.csv")
vacationDF.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Cloudiness,Wind Speed,Humidity
0,0,Rikitea,-23.12,-134.97,75.56,100,16.75,83
1,1,Pevek,69.70,170.31,20.44,100,13.73,97
2,2,Vaini,-21.20,-175.20,77.00,20,12.75,73
3,3,Visnes,59.36,5.22,48.38,100,13.87,87
4,4,Kapaa,22.08,-159.32,82.13,75,5.82,69


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)

In [14]:
locations = vacationDF[["Latitude", "Longitude"]]

humidity = vacationDF["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
cleanVacation = vacationDF.drop(vacationDF[vacationDF['Temperature']> 70].index, inplace = True) 
cleanVacation = vacationDF.drop(vacationDF[vacationDF['Temperature']< 80].index, inplace = True) 
cleanVacation = vacationDF.drop(vacationDF[vacationDF['Wind Speed']< 10].index, inplace = True) 
cleanVacation = vacationDF.drop(vacationDF[vacationDF['Cloudiness']== 0].index, inplace = True) 
cleanVacation = vacationDF.drop(vacationDF[vacationDF['Temperature']> 70].index, inplace = True) 

cleanVacation

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
